In [2]:
# !git clone https://github.com/ajay-sainy/Wav2Lip-GFPGAN.git
# basePath = "/content/Wav2Lip-GFPGAN"
basePath = "C:/Users/Administrator/Desktop/Wav2Lip-GFPGAN"
%cd {basePath}


C:\Users\Administrator\Desktop\Wav2Lip-GFPGAN


In [3]:
wav2lipFolderName = 'Wav2Lip-master'
gfpganFolderName = 'GFPGAN-master'
wav2lipPath = basePath + '/' + wav2lipFolderName
gfpganPath = basePath + '/' + gfpganFolderName

# !wget 'https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth' -O {wav2lipPath}'/face_detection/detection/sfd/s3fd.pth'
# !gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q --output {wav2lipPath}'/checkpoints/'


In [4]:
!pip install -r requirements.txt


Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ---------------------------------------- 0.0/52.2 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.2 kB ? eta -:--:--
     -------------- ----------------------- 20.5/52.2 kB 330.3 kB/s eta 0:00:01
     -------------- ----------------------- 20.5/52.2 kB 330.3 kB/s eta 0:00:01
     ----------------------------- -------- 41.0/52.2 kB 217.9 kB/s eta 0:00:01
     -------------------------------------- 52.2/52.2 kB 243.8 kB/s eta 0:00:00
  Attempting uninstall: gfpgan
    Found existing installation: gfpgan 1.3.4
    Uninstalling gfpgan-1.3.4:
      Successfully uninstalled gfpgan-1.3.4


In [5]:
import os
outputPath = basePath+'/outputs'
inputAudioPath = basePath + '/inputs/1111.wav'
inputVideoPath = basePath + '/inputs/test.mp4'
lipSyncedOutputPath = basePath + '/outputs/result.mp4'

if not os.path.exists(outputPath):
    os.makedirs(outputPath)

!cd $wav2lipFolderName && python inference.py \
    --checkpoint_path checkpoints/wav2lip.pth \
    --face {inputVideoPath} \
    --audio {inputAudioPath} \
    --outfile {lipSyncedOutputPath}


^C


In [6]:
!cd $gfpganFolderName && python setup.py develop
# !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth -P {gfpganFolderName}'/experiments/pretrained_models'


running develop
running egg_info
writing gfpgan.egg-info\PKG-INFO
writing dependency_links to gfpgan.egg-info\dependency_links.txt
writing requirements to gfpgan.egg-info\requires.txt
writing top-level names to gfpgan.egg-info\top_level.txt
reading manifest file 'gfpgan.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
adding license file 'LICENSE'
writing manifest file 'gfpgan.egg-info\SOURCES.txt'
running build_ext
Creating c:\miniconda3\envs\szr310\lib\site-packages\gfpgan.egg-link (link to .)
Adding gfpgan 1.3.4 to easy-install.pth file


C:\Miniconda3\envs\szr310\lib\site-packages\setuptools\__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
C:\Miniconda3\envs\szr310\lib\site-packages\setuptools\command\develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        *************

In [7]:
import cv2
from tqdm import tqdm
from os import path

import os

inputVideoPath = outputPath+'/result.mp4'
unProcessedFramesFolderPath = outputPath+'/frames'

if not os.path.exists(unProcessedFramesFolderPath):
    os.makedirs(unProcessedFramesFolderPath)

vidcap = cv2.VideoCapture(inputVideoPath)
numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = vidcap.get(cv2.CAP_PROP_FPS)
print("FPS: ", fps, "Frames: ", numberOfFrames)

for frameNumber in tqdm(range(numberOfFrames)):
    _, image = vidcap.read()
    cv2.imwrite(path.join(unProcessedFramesFolderPath,
                str(frameNumber).zfill(4)+'.jpg'), image)

  0%|          | 0/547 [00:00<?, ?it/s]

FPS:  30.0 Frames:  547


100%|██████████| 547/547 [00:21<00:00, 25.24it/s]


In [8]:
!cd $gfpganFolderName & & \
    python inference_gfpgan.py - i $unProcessedFramesFolderPath - o $outputPath - v 1.3 - s 2 - -only_center_face - -bg_upsampler None


此时不应有 &。


In [9]:
from tqdm import tqdm
import numpy as np
import cv2
import os
restoredFramesPath = outputPath + '/restored_imgs/'
processedVideoOutputPath = outputPath

dir_list = os.listdir(restoredFramesPath)
dir_list.sort()


batch = 0
batchSize = 300
for i in tqdm(range(0, len(dir_list), batchSize)):
    img_array = []
    start, end = i, i+batchSize
    print("processing ", start, end)
    for filename in tqdm(dir_list[start:end]):
        filename = restoredFramesPath+filename
        img = cv2.imread(filename)
        if img is None:
            continue
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)

    out = cv2.VideoWriter(processedVideoOutputPath+'/batch_' +
                          str(batch).zfill(4)+'.avi', cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
    batch = batch + 1

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

  0%|          | 0/300 [00:00<?, ?it/s]

processing  0 300



  0%|          | 1/247 [00:00<00:28,  8.56it/s]

processing  300 600



100%|██████████| 2/2 [01:52<00:00, 56.19s/it]


In [10]:
from google.colab import files
concatTextFilePath = outputPath + "/concat.txt"
concatTextFile = open(concatTextFilePath, "w")
for ips in range(batch):
    concatTextFile.write("file batch_" + str(ips).zfill(4) + ".avi\n")
concatTextFile.close()

concatedVideoOutputPath = outputPath + "/concated_output.avi"
!ffmpeg - y - f concat - i {concatTextFilePath} - c copy {concatedVideoOutputPath}

finalProcessedOuputVideo = processedVideoOutputPath+'/final_with_audio.avi'
!ffmpeg - y - i {concatedVideoOutputPath} - i {inputAudioPath} - map 0 - map 1: a - c: v copy - shortest {finalProcessedOuputVideo}

files.download(finalProcessedOuputVideo)


ModuleNotFoundError: No module named 'google.colab'